# 👤 DDL: Tabela Silver - `tb_cliente`

**Origem:** Tabela Bronze `clientes`
**Domínio:** Cadastro e Pessoas

## ⚙️ Decisões de Governança e Otimização

### 1. Tipagem e Performance
Realizamos *casting* explícito para reduzir o armazenamento e melhorar performance de *scans*:
- **Idade (`nu_idade`):** Convertido de `BIGINT` (Bronze) para `SMALLINT`. Economia de espaço, já que idade ≤ 150.
- **Campos de Texto:** `nm_cliente`, `ds_email` tipados como `VARCHAR` com limite definido (150/50) ao invés de String genérica.

### 2. Mapeamento de Schema (Bronze → Silver)
| Coluna Bronze | Coluna Silver | Tipo Novo | Justificativa |
|---|---|---|---|
| `id_cliente` | `cd_cliente` | `STRING` | PK Hash UUID. |
| `cpf` | `nu_cpf` | `STRING` | Padronização de prefixo numérico (`nu_`). |
| `nome` | `nm_cliente` | `VARCHAR(150)` | Padronização de prefixo nome (`nm_`). |
| `email` | `ds_email` | `VARCHAR(150)` | Padronização descritiva (`ds_`). |
| `idade` | `nu_idade` | `SMALLINT` | **Otimização de storage**. |

### 3. Constraints
- **Primary Key:** `pk_cliente` (`cd_cliente`).
- **Unique Keys (Observação):** O campo `ds_email` é marcado como UK no dicionário de negócio.

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import TimestampType

CATALOGO_DESTINO = "v_credit"
SCHEMA_DESTINO = "silver"
TABELA_DESTINO = "tb_cliente"

nome_tabela_destino = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_DESTINO}"

In [0]:
query = f"""
CREATE OR REPLACE TABLE {nome_tabela_destino} (
    cd_cliente STRING COMMENT 'Identificador único do cliente.',
    nu_cpf STRING COMMENT 'Numero do cpf do cliente.',
    nm_cliente VARCHAR(150) COMMENT 'Nome completo do cliente.',
    ds_email VARCHAR(150) COMMENT 'Endereço de e-mail.',
    nm_regiao VARCHAR(50) COMMENT 'Região geográfica do cliente.',
    nu_idade SMALLINT COMMENT 'Idade do cliente.',
    dt_ingestion TIMESTAMP,
    dc_origem STRING,
    CONSTRAINT pk_cliente PRIMARY KEY (cd_cliente)
)
COMMENT 'Tabela silver de cadastro de clientes';
"""

spark.sql(query)
print(f"✅ Tabela {nome_tabela_destino} criada com sucesso!")